In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from birdclef.kaggle.compile import compile_classifier_head, benchmark_perch

compile_classifier_head(
    "~/shared/birdclef/models/2025/compiled",
    "Perch",
    "~/shared/birdclef/models/2025/v1/Perch/torch-linear-v1/checkpoints/best-checkpoint-Perch-epoch=05-val_loss=0.76.ckpt",
)

Saved compiled model to /storage/home/hcoda1/8/amiyaguchi3/shared/birdclef/models/2025/compiled/Perch_head.onnx


In [27]:
# load this model and try it out
import openvino as ov
import numpy as np
from birdclef.config import model_config

model = ov.compile_model(
    Path("~/shared/birdclef/models/2025/compiled/Perch_head.onnx").expanduser(), "CPU"
)

# now run a prediction
example_input = np.random.rand(1, model_config["Perch"]["embed_size"]).astype(
    np.float32
)

res = model(example_input)
res[0].shape

(1, 204)

In [35]:
benchmark_perch(
    "~/scratch/birdclef/test_soundscape",
    "~/shared/birdclef/models/2025/compiled",
    "~/shared/birdclef/models/2025/v1/Perch/torch-linear-v1/checkpoints/best-checkpoint-Perch-epoch=05-val_loss=0.76.ckpt",
)

2025-06-06 03:34:17.868964: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_inter_op_parallelism which is not in the op definition: Op<name=Transpose; signature=x:T, perm:Tperm -> y:T; attr=T:type; attr=Tperm:type,default=DT_INT32,allowed=[DT_INT32, DT_INT64]> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node Transpose}}
/storage/home/hcoda1/8/amiyaguchi3/clef/birdclef-2025/.venv/lib/python3.10/site-packages/opensoundscape/ml/cnn.py:599: UserWarning: 
                    This architecture is not listed in opensoundscape.ml.cnn_architectures.ARCH_DICT.
                    It will not be available for loading after saving the model with .save() (unless using pickle=True). 
                    To make it re-loadable, define a function that generates the architecture from arguments: (n_classes, n_channels) 
                    then use opensoundscape.ml.cnn_architectures.regi

Running Perch TFLite model...
                                                                            0     ...      1279
file                                               start_time end_time            ...          
/storage/home/hcoda1/8/amiyaguchi3/scratch/bird... 0.0        5.0      -0.013070  ... -0.033343
                                                   5.0        10.0      0.050018  ... -0.031186
                                                   10.0       15.0      0.031905  ... -0.055638
                                                   15.0       20.0     -0.046089  ... -0.048108
                                                   20.0       25.0      0.078878  ... -0.018229

[5 rows x 1280 columns]
Perch TFLite took 6.54 seconds for 5 files
Running classification head
Classification head took 0.00 seconds for 5 files
OpenVINO classification head took 0.01 seconds for 5 files


Cool, with this we know that the classifier head on top is negligible in speed.
We effectively don't need to care about it.